# Implementation of Basic ACO

Note: The code in this notebook assumes that you have the functions to read a TSP file in a file called readTSP.py and returns the content as a dictionary

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from readTSP import *

### Randomly place the ants

In [ ]:
def initializeAnts(cities, colony):
    return np.random.randint(cities.shape[0], size = colony)

### Compute the inverse of distances between cities

This is used for the calculation of the heuristic values

In [ ]:
def inverseDistances(cities):
    # A 2-dimensional array of zeros
    distances = np.zeros((cities.shape[0], cities.shape[0]))

    # Calculate distance between nodes
    for index, point in enumerate(cities):
        distances[index] = np.sqrt(((cities - point) ** 2).sum(axis = 1))

    with np.errstate(all = 'ignore'):    # Floating-point error handling - Setted to known state
        inv_distances = 1 / distances    # invert distances
    inv_distances[inv_distances == np.inf] = 0    # Replace infinity by zero to prevent zero division error

    return inv_distances

In [ ]:
def moveAnts(cities, positions, inv_distances, pheromones, alpha, beta, del_tau):
    
    paths = np.zeros((cities.shape[0], positions.shape[0]), dtype = int) - 1

    # Initial position at node zero
    paths[0] = positions

    # For nodes after start to end
    for node in range(1, cities.shape[0]):
        # For each ant
        for ant in range(positions.shape[0]):
            # Probability to travel the nodes
            next_location_probability = (inv_distances[positions[ant]] ** alpha + pheromones[positions[ant]] ** beta /
                                            inv_distances[positions[ant]].sum() ** alpha + pheromones[positions[ant]].sum() ** beta)

            # Index to maximum probability node
            next_position = np.argwhere(next_location_probability == np.amax(next_location_probability))[0][0]

            # Check if node has already been visited
            while next_position in paths[:, ant]:
                # Replace the probability of visited to zero
                next_location_probability[next_position] = 0.0

                # Find the maximum probability node
                next_position = np.argwhere(next_location_probability == np.amax(next_location_probability))[0][0]

            # Add node to path
            paths[node, ant] = next_position

            # Update pheromones (releasing pheromones)
            pheromones[node, next_position] = pheromones[node, next_position] + del_tau

    # Paths taken by the ants
    return np.swapaxes(paths, 0, 1)

In [ ]:
def runAcoTsp(cities, iterations = 80, colony = 50, alpha = 1.0, beta = 1.0, del_tau = 1.0, rho = 0.5):
    
    inv_distances = inverseDistances(cities)
    inv_distances = inv_distances ** beta    # Add beta algorithm parameter to inverted distances

    # Initialize pheromones to zero, path to nothing
    pheromones = np.zeros((cities.shape[0], cities.shape[0]))
    min_distance = None
    min_path = None

    # For the number of iterations
    for i in range(iterations):
        positions = initializeAnts(cities, colony)    # Randomly place ants
        paths = moveAnts(cities, positions, inv_distances, pheromones, alpha, beta, del_tau)    # Find a path

        pheromones *= (1 - rho)    # Evaporate pheromones

        # [3] For each path
        for path in paths:
            distance = 0
            
            # For each node from second to last
            for node in range(1, path.shape[0]):
                # Calculate distance to the last node
                distance += np.sqrt(((cities[int(path[node])] - cities[int(path[node - 1])]) ** 2).sum())

            # Update minimun distance and path if less nor non existent
            if not min_distance or distance < min_distance:
                min_distance = distance
                min_path = path

    # Copy and append first node to end of minimum path to form closed path
    min_path = np.append(min_path, min_path[0])

    # Return tuple
    return (min_path, min_distance)

## Execute ACO for a TSP

In [ ]:
# Read data
TSP = getTspData('berlin52.tsp')
displayTspHeaders(TSP)
cities = np.array(TSP['node_coord_section'])

# Plot
plt.scatter(cities[:, 0], cities[:, 1], s = 15)
plt.title('Cities for {}'.format(TSP['name']))
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()
plt.close()

# ACO Parameters
iterations = 50
colony = 25
alpha = 1
beta = 1
del_tau = 1.5
rho = 0.5

# Vars
n = 2         # Run ACO this many times
average = 0 

# Repeat
for i in range(n):
    # Execute ACO
    min_path, min_distance = runAcoTsp(cities, iterations, colony, alpha, beta, del_tau, rho)
    average += min_distance
    
    # Plot solution
    plt.scatter(cities[:, 0], cities[:, 1], marker='o', s=15)
    plt.plot(cities[min_path, 0], cities[min_path, 1], c='g', linewidth=0.8, linestyle="--")
    plt.suptitle('Mininum Path for {}'.format(TSP['name']))
    plt.title('Result #{} of {} for a minimum distance of {}'.format(i + 1, n, min_distance), fontsize = 10)
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
        
    plt.show()
    plt.close()
    
# Show Average over n runs
print('Min Distance Average for the last {} results is {}'.format(n, average/n))
